In [63]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'ko_ice_reader',
                        'XXXXXXX',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res
    
def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [64]:
query = """ 

select distinct op.LocationId, op.UserId, op.DateId, op.IsReported, YEAR(srl.VisitStartDateTime) as year, MONTH(op.VisitStartDateTime) as month, DAY(op.VisitStartDateTime) as day, op.Segment, op.MarketSegment, op.ICEScore, sl.Question, sl.Response, srl.* from dbo.ICEOnPremise op 
join dbo.SurveyList sl on sl.SessionUid = op.SessionUId
join dbo.SceneReportList srl on srl.SessionUid = op.SessionUId
where YEAR(srl.VisitStartDateTime) = '2021' and srl.LocationClusterName = 'On-premise' and srl.UserRole = 'Auditor' and srl.Scene = 'Enfriador' and srl.SubSceneType in ('Enfriador Coca-Cola', 'Enfriador Competencia', 'Enfriador Detallista') 
 
"""

In [65]:
df = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
df

,LocationId,UserId,DateId,IsReported,year,month,day,Segment,MarketSegment,ICEScore,...,SurveyCancelCallReason,IsQAEnvironment,SalesOfficeCode,SalesOfficeName,IdVisita,SalesGroupCode,SalesGroupName,TradeGroupCode,TradeGroupName,ParentSceneUid
0,6208850,45794,20210119,1,2021,1,19,Chico,PUESTOS,75.0,...,,None,Yucatán,Yucatán,None,9000222-31,Bodega Poniente,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,
1,6227062,45800,20210111,1,2021,1,11,Chico,RSR,85.0,...,,None,Silao,Silao,None,MR0042,Silao,RSR TACOS,RSR TACOS,
2,6227062,45800,20210111,1,2021,1,11,Chico,RSR,85.0,...,,None,Silao,Silao,None,MR0042,Silao,RSR TACOS,RSR TACOS,
3,6208850,45794,20210119,1,2021,1,19,Chico,PUESTOS,75.0,...,,None,Yucatán,Yucatán,None,9000222-31,Bodega Poniente,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,
4,6227062,45800,20210111,1,2021,1,11,Chico,RSR,85.0,...,,None,Silao,Silao,None,MR0042,Silao,RSR TACOS,RSR TACOS,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618996,9812787,45937,20210514,1,2021,5,14,Chico,PUESTOS,100.0,...,,None,Tapachula,Tapachula,220153440tuxtla@storecheck.com2021-05-14,MR0180,Arriaga,PUESTO FIJO ALIMENTOS,PUESTO FIJO ALIMENTOS,
618997,9812741,45830,20210515,1,2021,5,15,Chico,FONDAS,100.0,...,,None,Teziutlán,Teziutlán,502100331veracruz@storecheck.com2021-05-15,MR0125,Teziutlán,FONDA/LONCHERÍA/MERENDERO,FONDA/LONCHERÍA/MERENDERO,
618998,9812917,45817,20210518,1,2021,5,18,Chico,PUESTOS,10.0,...,,None,Zihuatanejo,Zihuatanejo,610040675uruapan@storecheck.com2021-05-18,MR0071,Lázaro Cárdenas,PUESTO FIJO ALIMENTOS,PUESTO FIJO ALIMENTOS,
618999,9812959,54609,20210507,1,2021,5,7,Chico,PUESTOS,80.0,...,,None,Chalco,Chalco,105213209cdmx38@storecheck.com2021-05-07,MR0024,Chalco,PUESTO FIJO ALIMENTOS,PUESTO FIJO ALIMENTOS,


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619001 entries, 0 to 619000
Data columns (total 50 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   LocationId                    619001 non-null  int64         
 1   UserId                        619001 non-null  int64         
 2   DateId                        619001 non-null  int64         
 3   IsReported                    619001 non-null  int64         
 4   year                          619001 non-null  int64         
 5   month                         619001 non-null  int64         
 6   day                           619001 non-null  int64         
 7   Segment                       619001 non-null  object        
 8   MarketSegment                 619001 non-null  object        
 9   ICEScore                      619001 non-null  float64       
 10  Question                      619001 non-null  object        
 11  Response     

In [67]:
df['date_format']=pd.to_datetime(df[['year','month','day']])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619001 entries, 0 to 619000
Data columns (total 51 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   LocationId                    619001 non-null  int64         
 1   UserId                        619001 non-null  int64         
 2   DateId                        619001 non-null  int64         
 3   IsReported                    619001 non-null  int64         
 4   year                          619001 non-null  int64         
 5   month                         619001 non-null  int64         
 6   day                           619001 non-null  int64         
 7   Segment                       619001 non-null  object        
 8   MarketSegment                 619001 non-null  object        
 9   ICEScore                      619001 non-null  float64       
 10  Question                      619001 non-null  object        
 11  Response     

In [68]:
df.duplicated(subset='SessionUid', keep='first').sum()

590262

In [69]:
df=df.drop_duplicates(subset=['SessionUid'])
df

,LocationId,UserId,DateId,IsReported,year,month,day,Segment,MarketSegment,ICEScore,...,IsQAEnvironment,SalesOfficeCode,SalesOfficeName,IdVisita,SalesGroupCode,SalesGroupName,TradeGroupCode,TradeGroupName,ParentSceneUid,date_format
0,6208850,45794,20210119,1,2021,1,19,Chico,PUESTOS,75.0,...,None,Yucatán,Yucatán,None,9000222-31,Bodega Poniente,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,,2021-01-19
1,6227062,45800,20210111,1,2021,1,11,Chico,RSR,85.0,...,None,Silao,Silao,None,MR0042,Silao,RSR TACOS,RSR TACOS,,2021-01-11
7,6227062,45800,20210210,1,2021,2,10,Chico,RSR,85.0,...,None,Silao,Silao,600601107leon3@storecheck.com2021-02-10,MR0042,Silao,RSR TACOS,RSR TACOS,,2021-02-10
20,6227062,45800,20210318,1,2021,3,18,Chico,RSR,90.0,...,None,Silao,Silao,None,MR0042,Silao,RSR TACOS,RSR TACOS,,2021-03-18
35,6227062,45800,20210415,1,2021,4,15,Chico,RSR,90.0,...,None,Silao,Silao,600601107leon3@storecheck.com2021-04-15,MR0042,Silao,RSR TACOS,RSR TACOS,,2021-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588248,8595484,45796,20210208,1,2021,2,8,Mediano,PUESTOS,10.0,...,None,Yucatán,Yucatán,None,9000222-30,Oriente,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,,2021-02-08
588667,8595966,45788,20210202,1,2021,2,2,Chico,PUESTOS,90.0,...,None,Campeche,Campeche,None,9000222-13,Campeche Oriente,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,,2021-02-02
588896,8594858,45789,20210203,1,2021,2,3,Chico,PUESTOS,90.0,...,None,Quintana Roo,Quintana Roo,None,9000222-49,Cancún Planta,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,,2021-02-03
589921,9080796,45788,20210506,1,2021,5,6,Chico,PUESTOS,20.0,...,None,Campeche,Campeche,380972campeche@storecheck.com2021-05-06,9000222-76,Minibodegas IECSA,PUESTO FIJO DE VENTA DE ALIMENTOS,PUESTO FIJO DE VENTA DE ALIMENTOS,,2021-05-06


In [70]:
df.dtypes

LocationId                               int64
UserId                                   int64
DateId                                   int64
IsReported                               int64
year                                     int64
month                                    int64
day                                      int64
Segment                                 object
MarketSegment                           object
ICEScore                               float64
Question                                object
Response                                object
AssetPurityId                            int64
UserName                                object
OutletCode                              object
Outlet                                  object
ImageCaptureTime                datetime64[ns]
SubSceneType                            object
Scene                                   object
Status                                  object
SubStatus                               object
ClientCode   

In [73]:
table = pd.pivot_table(df, values='Status', index=['UserName', 'OutletCode', 'UserId', 'DateId', 'IsReported', 'date_format', 'month', 'day', 'Question', 'Response', 'SessionUid'],
                    columns=['SubSceneType'], aggfunc={'Status':len}, fill_value=0)
table=pd.DataFrame(table)
table

SubSceneType                                                                                                                                                                          Enfriador Coca-Cola  \
UserName                OutletCode UserId DateId   IsReported date_format month day Question                                           Response SessionUid                                                  
acambaro@storecheck.com 1601044547 45806  20210116 1          2021-01-16  1     16  ¿Existe algún producto que no pertenezca a la f... No      c2c69c7a-3d2f-4608-a166-b48a73efc254                    0   
                                          20210322 1          2021-03-22  3     22  ¿El enfriador tiene tapa ciega?                    No      c1bd7779-b92f-418d-9f75-9604568e65f4                    1   
                                          20210422 0          2021-04-22  4     22  ¿Cuenta con comunicación interior?                 Yes     64587182-dd48-4bf5-9cb0-42bcd1ef1764                    0   
                        1601074272 45806  20210120 1          2021-01-20  1     20  Foto de la fachada de la tienda                    Yes     3773476d-f3d7-4868-ba17-49de0cf3b9ae                    1   
                                          20210216 1          2021-02-16  2     16  ¿El enfriador se encuentra visible por lo menos... Yes     31c635dc-58c3-4622-9acd-3bf7949584c9                    1   
...                                                                                                                                                                                                   ...   
zamora@storecheck.com   620155472  45818  20210422 1          2021-04-22  4     22  Foto del código/sticker del cliente                Yes     1ca3904e-d6dd-44a9-8c22-7ab880beb0ce                    1   
                        620157827  45818  20210114 1          2021-01-14  1     14  Foto del código/sticker del cliente                Yes     7429bcfe-86b7-4174-a181-c9169811cd25                    1   
                                          20210211 1          2021-02-11  2     11  ¿Es un cliente exclusivo Coca Cola?                No      544077f8-3e4b-4bea-b8f9-c6695c67c2c0                    1   
                                          20210313 1          2021-03-13  3     13  ¿El cliente cuenta con enfriador de Coca-Cola?     Yes     d7cc94e7-4cf4-4194-9d60-6e81f190d4ef                    1   
                                          20210416 1          2021-04-16  4     16  ¿El cliente cuenta con enfriador de Coca-Cola?     Yes     cc963e44-1a62-48cd-980e-a1576bf14887                    1   

SubSceneType                                                                                                                                                                          Enfriador Competencia  \
UserName                OutletCode UserId DateId   IsReported date_format month day Question                                           Response SessionUid                                                    
acambaro@storecheck.com 1601044547 45806  20210116 1          2021-01-16  1     16  ¿Existe algún producto que no pertenezca a la f... No      c2c69c7a-3d2f-4608-a166-b48a73efc254                      0   
                                          20210322 1          2021-03-22  3     22  ¿El enfriador tiene tapa ciega?                    No      c1bd7779-b92f-418d-9f75-9604568e65f4                      0   
                                          20210422 0          2021-04-22  4     22  ¿Cuenta con comunicación interior?                 Yes     64587182-dd48-4bf5-9cb0-42bcd1ef1764                      0   
                        1601074272 45806  20210120 1          2021-01-20  1     20  Foto de la fachada de la tienda                    Yes     3773476d-f3d7-4868-ba17-49de0cf3b9ae                      0   
                                          20210216 1          2021-02-16  2     16  ¿El enfr

In [74]:
table = table.reset_index()
table

SubSceneType,UserName,OutletCode,UserId,DateId,IsReported,date_format,month,day,Question,Response,SessionUid,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista
0,acambaro@storecheck.com,1601044547,45806,20210116,1,2021-01-16,1,16,¿Existe algún producto que no pertenezca a la ...,No,c2c69c7a-3d2f-4608-a166-b48a73efc254,0,0,1
1,acambaro@storecheck.com,1601044547,45806,20210322,1,2021-03-22,3,22,¿El enfriador tiene tapa ciega?,No,c1bd7779-b92f-418d-9f75-9604568e65f4,1,0,0
2,acambaro@storecheck.com,1601044547,45806,20210422,0,2021-04-22,4,22,¿Cuenta con comunicación interior?,Yes,64587182-dd48-4bf5-9cb0-42bcd1ef1764,0,0,1
3,acambaro@storecheck.com,1601074272,45806,20210120,1,2021-01-20,1,20,Foto de la fachada de la tienda,Yes,3773476d-f3d7-4868-ba17-49de0cf3b9ae,1,0,0
4,acambaro@storecheck.com,1601074272,45806,20210216,1,2021-02-16,2,16,¿El enfriador se encuentra visible por lo meno...,Yes,31c635dc-58c3-4622-9acd-3bf7949584c9,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28734,zamora@storecheck.com,620155472,45818,20210422,1,2021-04-22,4,22,Foto del código/sticker del cliente,Yes,1ca3904e-d6dd-44a9-8c22-7ab880beb0ce,1,0,0
28735,zamora@storecheck.com,620157827,45818,20210114,1,2021-01-14,1,14,Foto del código/sticker del cliente,Yes,7429bcfe-86b7-4174-a181-c9169811cd25,1,0,0
28736,zamora@storecheck.com,620157827,45818,20210211,1,2021-02-11,2,11,¿Es un cliente exclusivo Coca Cola?,No,544077f8-3e4b-4bea-b8f9-c6695c67c2c0,1,0,0
28737,zamora@storecheck.com,620157827,45818,20210313,1,2021-03-13,3,13,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,d7cc94e7-4cf4-4194-9d60-6e81f190d4ef,1,0,0


In [75]:
table.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
month                             int64
day                               int64
Question                         object
Response                         object
SessionUid                       object
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
dtype: object

In [76]:
ifi=table.groupby(['UserName', 'OutletCode', 'UserId', 'DateId', 'IsReported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Competencia', 'Enfriador Detallista'].sum()

ifi  

<ipython-input-76-2dbf2535369e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ifi=table.groupby(['UserName', 'OutletCode', 'UserId', 'DateId', 'IsReported', 'date_format', 'SessionUid', 'Question', 'Response', 'month', 'day'])['Enfriador Coca-Cola', 'Enfriador Competencia', 'Enfriador Detallista'].sum()


SubSceneType                                                                                                                                                                          Enfriador Coca-Cola  \
UserName                OutletCode UserId DateId   IsReported date_format SessionUid                           Question                                           Response month day                        
acambaro@storecheck.com 1601044547 45806  20210116 1          2021-01-16  c2c69c7a-3d2f-4608-a166-b48a73efc254 ¿Existe algún producto que no pertenezca a la f... No      1     16                     0   
                                          20210322 1          2021-03-22  c1bd7779-b92f-418d-9f75-9604568e65f4 ¿El enfriador tiene tapa ciega?                    No      3     22                     1   
                                          20210422 0          2021-04-22  64587182-dd48-4bf5-9cb0-42bcd1ef1764 ¿Cuenta con comunicación interior?                 Yes     4     22                     0   
                        1601074272 45806  20210120 1          2021-01-20  3773476d-f3d7-4868-ba17-49de0cf3b9ae Foto de la fachada de la tienda                    Yes     1     20                     1   
                                          20210216 1          2021-02-16  31c635dc-58c3-4622-9acd-3bf7949584c9 ¿El enfriador se encuentra visible por lo menos... Yes     2     16                     1   
...                                                                                                                                                                                                   ...   
zamora@storecheck.com   620155472  45818  20210422 1          2021-04-22  1ca3904e-d6dd-44a9-8c22-7ab880beb0ce Foto del código/sticker del cliente                Yes     4     22                     1   
                        620157827  45818  20210114 1          2021-01-14  7429bcfe-86b7-4174-a181-c9169811cd25 Foto del código/sticker del cliente                Yes     1     14                     1   
                                          20210211 1          2021-02-11  544077f8-3e4b-4bea-b8f9-c6695c67c2c0 ¿Es un cliente exclusivo Coca Cola?                No      2     11                     1   
                                          20210313 1          2021-03-13  d7cc94e7-4cf4-4194-9d60-6e81f190d4ef ¿El cliente cuenta con enfriador de Coca-Cola?     Yes     3     13                     1   
                                          20210416 1          2021-04-16  cc963e44-1a62-48cd-980e-a1576bf14887 ¿El cliente cuenta con enfriador de Coca-Cola?     Yes     4     16                     1   

SubSceneType                                                                                                                                                                          Enfriador Competencia  \
UserName                OutletCode UserId DateId   IsReported date_format SessionUid                           Question                                           Response month day                          
acambaro@storecheck.com 1601044547 45806  20210116 1          2021-01-16  c2c69c7a-3d2f-4608-a166-b48a73efc254 ¿Existe algún producto que no pertenezca a la f... No      1     16                       0   
                                          20210322 1          2021-03-22  c1bd7779-b92f-418d-9f75-9604568e65f4 ¿El enfriador tiene tapa ciega?                    No      3     22                       0   
                                          20210422 0          2021-04-22  64587182-dd48-4bf5-9cb0-42bcd1ef1764 ¿Cuenta con comunicación interior?                 Yes     4     22                       0   
                        1601074272 45806  20210120 1          2021-01-20  3773476d-f3d7-4868-ba17-49de0cf3b9ae Foto de la fachada de la tienda                    Yes     1     20                       0   
                                          20210216 1          2021-02-16  31c635dc-58c3-4622

In [77]:
table2 = ifi.reset_index()
table2

SubSceneType,UserName,OutletCode,UserId,DateId,IsReported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista
0,acambaro@storecheck.com,1601044547,45806,20210116,1,2021-01-16,c2c69c7a-3d2f-4608-a166-b48a73efc254,¿Existe algún producto que no pertenezca a la ...,No,1,16,0,0,1
1,acambaro@storecheck.com,1601044547,45806,20210322,1,2021-03-22,c1bd7779-b92f-418d-9f75-9604568e65f4,¿El enfriador tiene tapa ciega?,No,3,22,1,0,0
2,acambaro@storecheck.com,1601044547,45806,20210422,0,2021-04-22,64587182-dd48-4bf5-9cb0-42bcd1ef1764,¿Cuenta con comunicación interior?,Yes,4,22,0,0,1
3,acambaro@storecheck.com,1601074272,45806,20210120,1,2021-01-20,3773476d-f3d7-4868-ba17-49de0cf3b9ae,Foto de la fachada de la tienda,Yes,1,20,1,0,0
4,acambaro@storecheck.com,1601074272,45806,20210216,1,2021-02-16,31c635dc-58c3-4622-9acd-3bf7949584c9,¿El enfriador se encuentra visible por lo meno...,Yes,2,16,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28734,zamora@storecheck.com,620155472,45818,20210422,1,2021-04-22,1ca3904e-d6dd-44a9-8c22-7ab880beb0ce,Foto del código/sticker del cliente,Yes,4,22,1,0,0
28735,zamora@storecheck.com,620157827,45818,20210114,1,2021-01-14,7429bcfe-86b7-4174-a181-c9169811cd25,Foto del código/sticker del cliente,Yes,1,14,1,0,0
28736,zamora@storecheck.com,620157827,45818,20210211,1,2021-02-11,544077f8-3e4b-4bea-b8f9-c6695c67c2c0,¿Es un cliente exclusivo Coca Cola?,No,2,11,1,0,0
28737,zamora@storecheck.com,620157827,45818,20210313,1,2021-03-13,d7cc94e7-4cf4-4194-9d60-6e81f190d4ef,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,13,1,0,0


In [78]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28739 entries, 0 to 28738
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   UserName               28739 non-null  object        
 1   OutletCode             28739 non-null  object        
 2   UserId                 28739 non-null  int64         
 3   DateId                 28739 non-null  int64         
 4   IsReported             28739 non-null  int64         
 5   date_format            28739 non-null  datetime64[ns]
 6   SessionUid             28739 non-null  object        
 7   Question               28739 non-null  object        
 8   Response               28739 non-null  object        
 9   month                  28739 non-null  int64         
 10  day                    28739 non-null  int64         
 11  Enfriador Coca-Cola    28739 non-null  int64         
 12  Enfriador Competencia  28739 non-null  int64         
 13  E

In [79]:
table2=table2.sort_values(['OutletCode', 'date_format'])

In [80]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28739 entries, 1942 to 15670
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   UserName               28739 non-null  object        
 1   OutletCode             28739 non-null  object        
 2   UserId                 28739 non-null  int64         
 3   DateId                 28739 non-null  int64         
 4   IsReported             28739 non-null  int64         
 5   date_format            28739 non-null  datetime64[ns]
 6   SessionUid             28739 non-null  object        
 7   Question               28739 non-null  object        
 8   Response               28739 non-null  object        
 9   month                  28739 non-null  int64         
 10  day                    28739 non-null  int64         
 11  Enfriador Coca-Cola    28739 non-null  int64         
 12  Enfriador Competencia  28739 non-null  int64         
 13

In [81]:
table2["Coca-Cola"] = table2["Enfriador Coca-Cola"].shift(1)
table2["Competencia"] = table2["Enfriador Competencia"].shift(1)
table2["Detallista"] = table2["Enfriador Detallista"].shift(1)
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                       float64
Competencia                     float64
Detallista                      float64
dtype: object

In [82]:
table2 = table2.fillna(0)
table2

SubSceneType,UserName,OutletCode,UserId,DateId,IsReported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista
1942,cdcuauhtemoc2@storecheck.com,10001273,53032,20210330,1,2021-03-30,76446d48-8bc8-4a3e-a0a5-e9023e8be8d2,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,30,1,0,0,0.0,0.0,0.0
1943,cdcuauhtemoc2@storecheck.com,10001273,53032,20210508,1,2021-05-08,2ef0081a-21db-44b8-be21-a44107d840aa,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,5,8,1,0,0,1.0,0.0,0.0
1944,cdcuauhtemoc2@storecheck.com,10001449,53032,20210223,1,2021-02-23,2ba6c950-4ce8-4ab7-af25-7b638a73bcc6,¿Cuenta con comunicación interior?,Yes,2,23,1,0,0,1.0,0.0,0.0
1945,cdcuauhtemoc2@storecheck.com,10001449,53032,20210325,1,2021-03-25,f43bcc24-003b-4415-89af-133db84abda4,¿Existe algún producto que no pertenezca a la ...,No,3,25,1,0,0,1.0,0.0,0.0
1946,cdcuauhtemoc2@storecheck.com,10001449,53032,20210419,1,2021-04-19,ddf9a83e-c6fe-4603-ab4c-d1c1e944ec36,¿Los manteles contienen marca?,Yes,4,19,1,0,0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18268,merida4@storecheck.com,9793,45796,20210212,1,2021-02-12,ee43b045-532b-4ea3-a4aa-69e9c61525e3,¿Cuenta con comunicación interior?,Yes,2,12,1,0,0,1.0,0.0,0.0
17886,merida2@storecheck.com,9793,45794,20210313,1,2021-03-13,df731564-cf6e-43cf-89d8-3a6153c7569c,¿El enfriador tiene colocado el sticker de No ...,No,3,13,1,0,0,1.0,0.0,0.0
17887,merida2@storecheck.com,9793,45794,20210413,1,2021-04-13,cb45c547-4c43-4c11-9c68-bf57b16d4bd7,¿Cuenta con comunicación interior?,Yes,4,13,1,0,0,1.0,0.0,0.0
18269,merida4@storecheck.com,9793,45796,20210511,1,2021-05-11,bb1f8ab2-dbc0-4ad4-b7b5-d78572ea8def,¿Cuál fue el problema?,Preguntas duplicadas o faltantes,5,11,1,0,0,1.0,0.0,0.0


In [83]:
table2 = table2.astype({"Coca-Cola": int, "Competencia": int, "Detallista": int})
table2

SubSceneType,UserName,OutletCode,UserId,DateId,IsReported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista
1942,cdcuauhtemoc2@storecheck.com,10001273,53032,20210330,1,2021-03-30,76446d48-8bc8-4a3e-a0a5-e9023e8be8d2,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,30,1,0,0,0,0,0
1943,cdcuauhtemoc2@storecheck.com,10001273,53032,20210508,1,2021-05-08,2ef0081a-21db-44b8-be21-a44107d840aa,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,5,8,1,0,0,1,0,0
1944,cdcuauhtemoc2@storecheck.com,10001449,53032,20210223,1,2021-02-23,2ba6c950-4ce8-4ab7-af25-7b638a73bcc6,¿Cuenta con comunicación interior?,Yes,2,23,1,0,0,1,0,0
1945,cdcuauhtemoc2@storecheck.com,10001449,53032,20210325,1,2021-03-25,f43bcc24-003b-4415-89af-133db84abda4,¿Existe algún producto que no pertenezca a la ...,No,3,25,1,0,0,1,0,0
1946,cdcuauhtemoc2@storecheck.com,10001449,53032,20210419,1,2021-04-19,ddf9a83e-c6fe-4603-ab4c-d1c1e944ec36,¿Los manteles contienen marca?,Yes,4,19,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18268,merida4@storecheck.com,9793,45796,20210212,1,2021-02-12,ee43b045-532b-4ea3-a4aa-69e9c61525e3,¿Cuenta con comunicación interior?,Yes,2,12,1,0,0,1,0,0
17886,merida2@storecheck.com,9793,45794,20210313,1,2021-03-13,df731564-cf6e-43cf-89d8-3a6153c7569c,¿El enfriador tiene colocado el sticker de No ...,No,3,13,1,0,0,1,0,0
17887,merida2@storecheck.com,9793,45794,20210413,1,2021-04-13,cb45c547-4c43-4c11-9c68-bf57b16d4bd7,¿Cuenta con comunicación interior?,Yes,4,13,1,0,0,1,0,0
18269,merida4@storecheck.com,9793,45796,20210511,1,2021-05-11,bb1f8ab2-dbc0-4ad4-b7b5-d78572ea8def,¿Cuál fue el problema?,Preguntas duplicadas o faltantes,5,11,1,0,0,1,0,0


In [84]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
dtype: object

In [85]:
table2["OutletCodeChanged"] = table2["OutletCode"].shift(1, fill_value=table2["OutletCode"].head(1)) != table2["OutletCode"]
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                  bool
dtype: object

In [86]:
table2

SubSceneType,UserName,OutletCode,UserId,DateId,IsReported,date_format,SessionUid,Question,Response,month,day,Enfriador Coca-Cola,Enfriador Competencia,Enfriador Detallista,Coca-Cola,Competencia,Detallista,OutletCodeChanged
1942,cdcuauhtemoc2@storecheck.com,10001273,53032,20210330,1,2021-03-30,76446d48-8bc8-4a3e-a0a5-e9023e8be8d2,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,30,1,0,0,0,0,0,False
1943,cdcuauhtemoc2@storecheck.com,10001273,53032,20210508,1,2021-05-08,2ef0081a-21db-44b8-be21-a44107d840aa,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,5,8,1,0,0,1,0,0,False
1944,cdcuauhtemoc2@storecheck.com,10001449,53032,20210223,1,2021-02-23,2ba6c950-4ce8-4ab7-af25-7b638a73bcc6,¿Cuenta con comunicación interior?,Yes,2,23,1,0,0,1,0,0,True
1945,cdcuauhtemoc2@storecheck.com,10001449,53032,20210325,1,2021-03-25,f43bcc24-003b-4415-89af-133db84abda4,¿Existe algún producto que no pertenezca a la ...,No,3,25,1,0,0,1,0,0,False
1946,cdcuauhtemoc2@storecheck.com,10001449,53032,20210419,1,2021-04-19,ddf9a83e-c6fe-4603-ab4c-d1c1e944ec36,¿Los manteles contienen marca?,Yes,4,19,1,0,0,1,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18268,merida4@storecheck.com,9793,45796,20210212,1,2021-02-12,ee43b045-532b-4ea3-a4aa-69e9c61525e3,¿Cuenta con comunicación interior?,Yes,2,12,1,0,0,1,0,0,False
17886,merida2@storecheck.com,9793,45794,20210313,1,2021-03-13,df731564-cf6e-43cf-89d8-3a6153c7569c,¿El enfriador tiene colocado el sticker de No ...,No,3,13,1,0,0,1,0,0,False
17887,merida2@storecheck.com,9793,45794,20210413,1,2021-04-13,cb45c547-4c43-4c11-9c68-bf57b16d4bd7,¿Cuenta con comunicación interior?,Yes,4,13,1,0,0,1,0,0,False
18269,merida4@storecheck.com,9793,45796,20210511,1,2021-05-11,bb1f8ab2-dbc0-4ad4-b7b5-d78572ea8def,¿Cuál fue el problema?,Preguntas duplicadas o faltantes,5,11,1,0,0,1,0,0,False


In [87]:
table2["OutletCodeChanged"] = table2["OutletCodeChanged"].astype(object)
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
dtype: object

In [88]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_CC = 'change'
              
          else: 
              Flag_CC = table2["Enfriador Coca-Cola"] - table2["Coca-Cola"]

              table2['Flag_CC']=Flag_CC
              
      print(Flag_CC)   

1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     1
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269  

In [89]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
Flag_CC                           int64
dtype: object

In [90]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Comp = 'change'
              
          else: 
              Flag_Comp = table2["Enfriador Competencia"] - table2["Competencia"]

              table2['Flag_Comp']=Flag_Comp
              
      print(Flag_Comp)  

1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269  

In [91]:
table2.dtypes

SubSceneType
UserName                         object
OutletCode                       object
UserId                            int64
DateId                            int64
IsReported                        int64
date_format              datetime64[ns]
SessionUid                       object
Question                         object
Response                         object
month                             int64
day                               int64
Enfriador Coca-Cola               int64
Enfriador Competencia             int64
Enfriador Detallista              int64
Coca-Cola                         int32
Competencia                       int32
Detallista                        int32
OutletCodeChanged                object
Flag_CC                           int64
Flag_Comp                         int64
dtype: object

In [92]:
for OutletCodeChanged in table2:
      s = table2['OutletCodeChanged'].unique()
      for i in s:
          if i == 'True':
              Flag_Det = 'change'
              
          else: 
              Flag_Det = table2["Enfriador Detallista"] - table2["Detallista"]

              table2['Flag_Det']=Flag_Det
              
      print(Flag_Det)   

1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269    0
15670    0
Length: 28739, dtype: int64
1942     0
1943     0
1944     0
1945     0
1946     0
        ..
18268    0
17886    0
17887    0
18269  

In [93]:
table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28739 entries, 1942 to 15670
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   UserName               28739 non-null  object        
 1   OutletCode             28739 non-null  object        
 2   UserId                 28739 non-null  int64         
 3   DateId                 28739 non-null  int64         
 4   IsReported             28739 non-null  int64         
 5   date_format            28739 non-null  datetime64[ns]
 6   SessionUid             28739 non-null  object        
 7   Question               28739 non-null  object        
 8   Response               28739 non-null  object        
 9   month                  28739 non-null  int64         
 10  day                    28739 non-null  int64         
 11  Enfriador Coca-Cola    28739 non-null  int64         
 12  Enfriador Competencia  28739 non-null  int64         
 13

In [94]:
table_final = table2[['UserName','UserId','OutletCode','DateId', 'IsReported', 'SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Coca-Cola', 'Flag_CC', 'Enfriador Competencia', 'Competencia', 'Flag_Comp', 'Enfriador Detallista', 'Detallista', 'Flag_Det']]
table_final

SubSceneType,UserName,UserId,OutletCode,DateId,IsReported,SessionUid,Question,Response,month,day,OutletCodeChanged,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Competencia,Competencia,Flag_Comp,Enfriador Detallista,Detallista,Flag_Det
1942,cdcuauhtemoc2@storecheck.com,53032,10001273,20210330,1,76446d48-8bc8-4a3e-a0a5-e9023e8be8d2,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,30,False,1,0,1,0,0,0,0,0,0
1943,cdcuauhtemoc2@storecheck.com,53032,10001273,20210508,1,2ef0081a-21db-44b8-be21-a44107d840aa,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,5,8,False,1,1,0,0,0,0,0,0,0
1944,cdcuauhtemoc2@storecheck.com,53032,10001449,20210223,1,2ba6c950-4ce8-4ab7-af25-7b638a73bcc6,¿Cuenta con comunicación interior?,Yes,2,23,True,1,1,0,0,0,0,0,0,0
1945,cdcuauhtemoc2@storecheck.com,53032,10001449,20210325,1,f43bcc24-003b-4415-89af-133db84abda4,¿Existe algún producto que no pertenezca a la ...,No,3,25,False,1,1,0,0,0,0,0,0,0
1946,cdcuauhtemoc2@storecheck.com,53032,10001449,20210419,1,ddf9a83e-c6fe-4603-ab4c-d1c1e944ec36,¿Los manteles contienen marca?,Yes,4,19,False,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18268,merida4@storecheck.com,45796,9793,20210212,1,ee43b045-532b-4ea3-a4aa-69e9c61525e3,¿Cuenta con comunicación interior?,Yes,2,12,False,1,1,0,0,0,0,0,0,0
17886,merida2@storecheck.com,45794,9793,20210313,1,df731564-cf6e-43cf-89d8-3a6153c7569c,¿El enfriador tiene colocado el sticker de No ...,No,3,13,False,1,1,0,0,0,0,0,0,0
17887,merida2@storecheck.com,45794,9793,20210413,1,cb45c547-4c43-4c11-9c68-bf57b16d4bd7,¿Cuenta con comunicación interior?,Yes,4,13,False,1,1,0,0,0,0,0,0,0
18269,merida4@storecheck.com,45796,9793,20210511,1,bb1f8ab2-dbc0-4ad4-b7b5-d78572ea8def,¿Cuál fue el problema?,Preguntas duplicadas o faltantes,5,11,False,1,1,0,0,0,0,0,0,0


In [95]:
table_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28739 entries, 1942 to 15670
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UserName               28739 non-null  object
 1   UserId                 28739 non-null  int64 
 2   OutletCode             28739 non-null  object
 3   DateId                 28739 non-null  int64 
 4   IsReported             28739 non-null  int64 
 5   SessionUid             28739 non-null  object
 6   Question               28739 non-null  object
 7   Response               28739 non-null  object
 8   month                  28739 non-null  int64 
 9   day                    28739 non-null  int64 
 10  OutletCodeChanged      28739 non-null  object
 11  Enfriador Coca-Cola    28739 non-null  int64 
 12  Coca-Cola              28739 non-null  int32 
 13  Flag_CC                28739 non-null  int64 
 14  Enfriador Competencia  28739 non-null  int64 
 15  Competencia     

In [96]:
query = """ 

select distinct sl.SessionUId, sl.LocationId from dbo.SessionList sl
where YEAR(sl.VisitStartDateTime) = '2021' 
 

        """

In [97]:
dff = pd.DataFrame(get_query(query))
#df = df.sort_values(by="VisitStartDateTime",ascending=False)
dff=dff.rename(columns={"SessionUId": "SessionUid"})
dff.dtypes

SessionUid    object
LocationId     int64
dtype: object

In [98]:
Ifi_historico = pd.merge(table_final, dff, on='SessionUid', how='inner')
Ifi_historico

,UserName,UserId,OutletCode,DateId,IsReported,SessionUid,Question,Response,month,day,...,Enfriador Coca-Cola,Coca-Cola,Flag_CC,Enfriador Competencia,Competencia,Flag_Comp,Enfriador Detallista,Detallista,Flag_Det,LocationId
0,cdcuauhtemoc2@storecheck.com,53032,10001273,20210330,1,76446d48-8bc8-4a3e-a0a5-e9023e8be8d2,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,3,30,...,1,0,1,0,0,0,0,0,0,9080605
1,cdcuauhtemoc2@storecheck.com,53032,10001273,20210508,1,2ef0081a-21db-44b8-be21-a44107d840aa,¿El cliente cuenta con enfriador de Coca-Cola?,Yes,5,8,...,1,1,0,0,0,0,0,0,0,9080605
2,cdcuauhtemoc2@storecheck.com,53032,10001449,20210223,1,2ba6c950-4ce8-4ab7-af25-7b638a73bcc6,¿Cuenta con comunicación interior?,Yes,2,23,...,1,1,0,0,0,0,0,0,0,6576428
3,cdcuauhtemoc2@storecheck.com,53032,10001449,20210325,1,f43bcc24-003b-4415-89af-133db84abda4,¿Existe algún producto que no pertenezca a la ...,No,3,25,...,1,1,0,0,0,0,0,0,0,6576428
4,cdcuauhtemoc2@storecheck.com,53032,10001449,20210419,1,ddf9a83e-c6fe-4603-ab4c-d1c1e944ec36,¿Los manteles contienen marca?,Yes,4,19,...,1,1,0,0,0,0,0,0,0,6576428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28734,merida4@storecheck.com,45796,9793,20210212,1,ee43b045-532b-4ea3-a4aa-69e9c61525e3,¿Cuenta con comunicación interior?,Yes,2,12,...,1,1,0,0,0,0,0,0,0,6475795
28735,merida2@storecheck.com,45794,9793,20210313,1,df731564-cf6e-43cf-89d8-3a6153c7569c,¿El enfriador tiene colocado el sticker de No ...,No,3,13,...,1,1,0,0,0,0,0,0,0,6475795
28736,merida2@storecheck.com,45794,9793,20210413,1,cb45c547-4c43-4c11-9c68-bf57b16d4bd7,¿Cuenta con comunicación interior?,Yes,4,13,...,1,1,0,0,0,0,0,0,0,6475795
28737,merida4@storecheck.com,45796,9793,20210511,1,bb1f8ab2-dbc0-4ad4-b7b5-d78572ea8def,¿Cuál fue el problema?,Preguntas duplicadas o faltantes,5,11,...,1,1,0,0,0,0,0,0,0,6475795


In [99]:
Ifi_historico.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28739 entries, 0 to 28738
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UserName               28739 non-null  object
 1   UserId                 28739 non-null  int64 
 2   OutletCode             28739 non-null  object
 3   DateId                 28739 non-null  int64 
 4   IsReported             28739 non-null  int64 
 5   SessionUid             28739 non-null  object
 6   Question               28739 non-null  object
 7   Response               28739 non-null  object
 8   month                  28739 non-null  int64 
 9   day                    28739 non-null  int64 
 10  OutletCodeChanged      28739 non-null  object
 11  Enfriador Coca-Cola    28739 non-null  int64 
 12  Coca-Cola              28739 non-null  int32 
 13  Flag_CC                28739 non-null  int64 
 14  Enfriador Competencia  28739 non-null  int64 
 15  Competencia        

In [100]:
Ifi_historico=Ifi_historico[['UserName', 'UserId', 'OutletCode', 'LocationId', 'DateId', 'IsReported','SessionUid', 'Question', 'Response', 'month', 'day', 'OutletCodeChanged', 'Enfriador Coca-Cola', 'Flag_CC', 'Enfriador Competencia', 'Flag_Comp', 'Enfriador Detallista', 'Flag_Det']]

In [102]:
Ifi_historico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28739 entries, 0 to 28738
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   UserName               28739 non-null  object
 1   UserId                 28739 non-null  int64 
 2   OutletCode             28739 non-null  object
 3   LocationId             28739 non-null  int64 
 4   DateId                 28739 non-null  int64 
 5   IsReported             28739 non-null  int64 
 6   SessionUid             28739 non-null  object
 7   Question               28739 non-null  object
 8   Response               28739 non-null  object
 9   month                  28739 non-null  int64 
 10  day                    28739 non-null  int64 
 11  OutletCodeChanged      28739 non-null  object
 12  Enfriador Coca-Cola    28739 non-null  int64 
 13  Flag_CC                28739 non-null  int64 
 14  Enfriador Competencia  28739 non-null  int64 
 15  Flag_Comp          

In [103]:
import pandas as pd 
import pyodbc
import sqlalchemy
import datetime as dt 
import numpy as np

#Se definen las conexiones

def get_engine():
    connection_string = "mssql+pyodbc://{0}@{2}:{1}@{2}.database.windows.net:{4}/{3}?driver=ODBC+Driver+17+for+SQL+Server".format(
                        'jrivas',
                        '8khHAM=B=q$ZGr6N',
                        'koice',
                        'ko_ice',
                        1433)
    res = sqlalchemy.engine.create_engine(connection_string, connect_args={'autocommit': True})
    return res


def get_query(query):
    engine = get_engine()
    res = pd.read_sql(query, con=engine)
    return res

In [104]:
engine = get_engine()
Ifi_historico.to_sql('Enfriadores_op', con=engine, schema='reportes', if_exists='append', index=False)